In [326]:
#https://open.kattis.com/problems/ads
#IN PROGRESS
import re
import os
# os.chdir("/Users/amfaber/Google Drive/Kattis/Ads data")
def loader(filename = None):
    path = ""
    if filename:
        filename = path + filename
        file = open(filename)
        reader = lambda: file.readline().rstrip("\n")
    else:
        reader = input
    
    dims = [int(x) for x in reader().split(" ")]
    lines = [None]*dims[0]
    for i in range(dims[0]):
        lines[i] = [x for x in reader()]
        
        
    if filename:
        file.close()
        
    return dims, lines

def getimgdims(lines, x, y, dims = None):
    if dims is None:
        dims = len(lines), len(lines[0])
    i = x + 1
    j = y + 1
    running = True
    while running:
        try:
            if j + 1 == dims[1]:
                running = False
                break
            elif lines[x][j + 1] == "+":
                j += 1
            else:
                running = False
        except IndexError as e:
            print(j + 1, dims[1])
            print(lines)
            raise e
    running = True
    while running:
        if i + 1 == dims[0]:
            running = False
            break
        elif lines[i + 1][y] == "+":
            i += 1
        else:
            running = False
    return i, j

def makewhitespace(lines, point1, point2):
    for i in range(point1[0], point2[0] + 1):
        for j in range(point1[1], point2[1] + 1):
            lines[i][j] = " "
            
def searchimg(lines, point1, point2):
    activeimgs = []
    for i in range(point1[0] + 1, point2[0]):
        for j in range(point1[1] + 1, point2[1]):
            skip = False
            for img in activeimgs:
                if img[0][0] <= i <= img[1][0] and img[0][1] <= j <= img[1][1]:
                    skip = True
                elif i == img[1][0] and j == img[1][1]:
                    activeimgs.remove(img)
            if skip:
                continue
            char = lines[i][j]
            if char == "+":
                p1 = i,j
                p2 = getimgdims(lines, i, j)
                activeimgs.append((p1, p2))
                searchimg(lines, p1, p2)
            elif not re.match(r"[a-zA-Z0-9\?\!\,\.\ ]", char):
                makewhitespace(lines, point1, point2)
                return None

def ini(lines):
    activeimgs = [] 
    for i, line in enumerate(lines):
        for j, char in enumerate(line):
            skip = False
            for img in activeimgs:
                if img[0][0] <= i <= img[1][0] and img[0][1] <= j <= img[1][1]:
                    skip = True
                elif i == img[1][0] and j == img[1][1]:
                    activeimgs.remove(img)
            if skip:
                continue
            if char == "+":
                p1 = i, j
                p2 = getimgdims(lines, *p1)
                activeimgs.append((p1, p2))
                searchimg(lines, p1, p2)
    return lines

printer = lambda l: "\n".join(["".join(line) for line in l])

def kattis(fn = None):
    dims, lines = loader(fn)
    lines = ini(lines)
    print(printer(lines))
kattis()

7 7
+++++++
+  =  +
+ +++ +
+ + + +
+ +++ +
+     +
+++++++
       
       
       
       
       
       
       


In [325]:
kattis("ads sample4.txt")

  apples are great!     apples are great! 
                      ++++++++++++++++++++
                      + oranges are      +
                      + way better       +
                      + orangesapples    +
                      ++++++++++++++++++++
  check out our        check out our      
  fresh apples!        fresh apples!      


In [ ]:
7 7
+++++++
+  =  +
+ +++ +
+ + + +
+ +++ +
+     +
+++++++

In [346]:
createdata()
dims, lines = loader("mydata.txt")
print("\n".join(["".join(line) for line in lines]))
print()
lines = ini(lines)
lines = ["".join(line) for line in lines]
lines = "\n".join(lines)
print(lines)

....................................................................................................
.................................+++++++++++++++++..................................................
.................................+...............+..................................................
.................................+...............+..................................................
....................++++++++.....+...............+..................................................
....................+......+.....+...............+..................................................
....................+......+.....+...............+.................++++++++++++++++++++.............
....................+......+.....+...............+.................+..................+.............
....................+......+.....+...............+.................+..................+.............
....................+......+.....+...............+.................+..................+....

In [262]:
import random
def createimg(page, p1, p2, ad):
    for i in range(p1[0], p2[0] + 1):
        page[i][p1[1]] = "+"
        page[i][p2[1]] = "+"
    for i in range(p1[1], p2[1] + 1):
        page[p1[0]][i] = "+"
        page[p2[0]][i] = "+"
    if ad:
        x, y = random.randint(p1[0]+1, p2[0]-1), random.randint(p1[1]+1, p2[1]-1)
        page[x][y] = "#"

In [270]:
R, C = 40, 100
page = [["." for j in range(C)] for i in range(R)]
imgs = []
checkclash()

In [280]:
checkclash(((0,0), (10, 10)), ((2,2), (8,8)))

True

In [263]:
def makecorners(p1, p4):
    p2 = p1[0], p4[1]
    p3 = p4[0], p1[1]
    return p1, p2, p3, p4

In [281]:
def checkclash(img1, img2):
    valid = False
    diffs = []
    for i in range(2):
        valid = valid or (img1[1][i] < img2[0][i] - 1) or (img1[0][i] > img2[1][i] + 1)
        
        diffs.append([(img1[0][i] - img2[0][i]), (img1[1][i] - img2[1][i])])
    
    valid = valid or (((diffs[0][0] * diffs[1][0]) > 0) and ((diffs[0][1] * diffs[1][1]) > 0)
                      and (diffs[0][0] * diffs[0][1] < 0))
    
    valid = valid and all([abs(d)>1 for diff in diffs for d in diff])
    
    return valid

In [282]:
def createdata(R = 40, C = 100, fn = "mydata.txt"):
    page = [["." for j in range(C)] for i in range(R)]
    imgs = []
    for i in range(100):
        rows = sorted([random.randint(0, R-1), random.randint(0, R-1)])
        cols = sorted([random.randint(0, C-1), random.randint(0, C-1)])
        if not rows[1] - rows[0] > 3:
            continue
        if not cols[1] - cols[0] > 3:
            continue
        p1, p2 = (rows[0], cols[0]), (rows[1], cols[1])

        valid = True
        for img in imgs:
            if not checkclash(img, (p1, p2)):
                valid = False
                break
        if valid:
            createimg(page, p1, p2, random.randint(0, 1))
            imgs.append((p1, p2))

    with open(fn, "w+") as file:
        file.write(f"{R} {C}\n")
        file.write(printer(page))
    